In [ ]:
from pathlib import Path
import pandas as pd
import xarray as xr

In [ ]:
file_list = list(Path('Archive').rglob('*.csv'))
file_list.sort()

In [ ]:
START = 0
if Path('cnemc.h5').exists():
    ds_archive = xr.open_dataset('cnemc.h5')
    START = len(ds_archive['timepoint'])

### 合并数据

In [ ]:
def convert_xarray(df):
    # df.set_index(['stationcode', 'timepoint'], inplace=True)
    ds = df.to_xarray()
    # ds = ds.set_coords(['longitude', 'latitude',
    #                     'area', 'positionname', 'primarypollutant'])
    return ds

In [ ]:
d_list = []
if START == 0:
    ds = xr.Dataset()
else:
    ds = ds_archive
# df_all = pd.DataFrame()

for f in file_list[START:]:
    df = pd.read_csv(f, parse_dates=['timepoint'])
    df.drop_duplicates(inplace=True)
    if df.duplicated(subset='stationcode').sum() > 0:
        d_list.append(f)
    # else:
    #     # df_all = df_all.append(df)
    #     ds_tmp = convert_xarray(df)
    #     ds = ds.merge(ds_tmp)

### 处理有重复的数据

In [ ]:
for d in d_list[:]:
    print(d)
    df = pd.read_csv(d, parse_dates=['timepoint'])
    df.drop_duplicates(inplace=True)
    print(df[df.duplicated(subset='stationcode', keep=False)].T)
    # 手动进行修改出问题的行，后期可以会换成下面的代码
    # df.drop_duplicates(subset='stationcode', keep='last')

### 导出文件

In [ ]:
ds['co'] = ds['co']*1000
ds['co_24h'] = ds['co_24h']*1000

ds['latitude'].attrs = {'units': 'degree_north'}
ds['longitude'].attrs = {'units': 'degree_east'}
ds.attrs = {'units': '除AQI无量纲，其他均是微克每立方米'}

In [ ]:
# 选取需要保存的的变量
selc_vars = list(ds.keys())[5:]

# 构造需要进行压缩的词典
enco_vars = {}
for d in selc_vars:
    enco_vars[d] = {"zlib": True,
                    "complevel": 9,
                    "dtype": "uint16",
                    '_FillValue': 65535
                    }

if START == 0:
    ds[selc_vars].to_netcdf('cnemc.h5',
                            engine='netcdf4',
                            encoding=enco_vars,
                            mode='w')
else:
    ds[selc_vars].to_netcdf('cnemc.h5',
                            engine='netcdf4',
                            encoding=enco_vars,
                            mode='a')

### 检查服务器上的历史数据

In [ ]:
df13 = pd.read_csv('/home/qin/Documents/CNEMC/Archive/2022-01-10/2022-01-10T20.csv',
                   parse_dates=['timepoint'])
df.drop_duplicates(inplace=True)
df.duplicated(subset='stationcode').sum() > 0